
Tugas

    1. Lakukan percobaan penggunaan ANNOY, FAISS, dan HNSWLIB pada dataset sekunder berukuran besar (Micro Spotify) pada link berikut: https://www.kaggle.com/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics/data
    2. Download data dan load CSV filenya (pilih dataset yang pertama dari dua dataset)
    3. Pilih hanya fitur numerik saja, dan lakukan normalisasi menggunakan StandardScaler
    4. Lakukan pencarian track terdekat dan bandingkan hasilnya



In [2]:
pip install annoy faiss-cpu hnswlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 85.5 MB/s eta 0:00:00
  Created wheel for annoy: filename=annoy-1.17.3-cp312-cp312-linux_x86_64.whl size=551809 sha256=33f0aa8b7c24138765dc47f238a0a481c39b00d0b5e9067661e74a73ca2e57e0
  Stored in directory: /root/.cache/pip/wheels/db/b9/53/a3b2d1fe1743abadddec6aa541294b24fdbc39d7800bc57311
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2528143 sha256=0beb888cd806ef332341b817892347689c2c17c2483dbf9ebbdc4e4e7e89e260
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built annoy hnswlib


In [3]:
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

import faiss
from annoy import AnnoyIndex
import hnswlib
import kagglehub


# ---------------------------------------------------------
# Utility: Benchmark Wrapper
# ---------------------------------------------------------
def timer(fn, *args, **kwargs):
    t0 = time.time()
    output = fn(*args, **kwargs)
    return output, time.time() - t0


# ---------------------------------------------------------
# Load Dataset
# ---------------------------------------------------------
print("Downloading dataset...")
dataset_path = kagglehub.dataset_download(
    "bwandowando/spotify-songs-with-attributes-and-lyrics"
)

csv_path = dataset_path + "/songs_with_attributes_and_lyrics.csv"
df = pd.read_csv(csv_path)

numeric_cols = [
    "danceability", "energy", "loudness", "speechiness",
    "acousticness", "instrumentalness", "liveness",
    "valence", "tempo"
]

X = df[numeric_cols].values.astype("float32")

# ---------------------------------------------------------
# Preprocessing
# ---------------------------------------------------------
scaler = StandardScaler()
X_norm = scaler.fit_transform(X).astype("float32")

k = 10  # jumlah tetangga


# ---------------------------------------------------------
# 1. Exact Nearest Neighbor (Brute Force)
# ---------------------------------------------------------
def exact_knn(data, k):
    model = NearestNeighbors(n_neighbors=k, algorithm="brute", metric="euclidean")
    model.fit(data)
    dist, idx = model.kneighbors(data)
    return dist, idx


(exact_res, exact_idx), t_exact = timer(exact_knn, X_norm, k)
print(f"[Exact] Waktu: {t_exact:.3f} s")


# ---------------------------------------------------------
# 2. Annoy
# ---------------------------------------------------------
def annoy_knn(data, k, n_trees=20):
    dim = data.shape[1]
    index = AnnoyIndex(dim, metric="euclidean")

    for i, vec in enumerate(data):
        index.add_item(i, vec)
    index.build(n_trees)

    neighbors = [index.get_nns_by_vector(vec, k) for vec in data]
    return neighbors


annoy_idx, t_ann = timer(annoy_knn, X_norm, k)
print(f"[Annoy] Waktu: {t_ann:.3f} s")


# ---------------------------------------------------------
# 3. HNSWLIB
# ---------------------------------------------------------
def hnsw_knn(data, k, ef=200, M=16):
    dim = data.shape[1]
    hnsw = hnswlib.Index(space="l2", dim=dim)
    hnsw.init_index(max_elements=len(data), ef_construction=ef, M=M)

    hnsw.add_items(data)
    hnsw.set_ef(ef)

    labels, dist = hnsw.knn_query(data, k)
    return labels


(hnsw_idx), t_hnsw = timer(hnsw_knn, X_norm, k)
print(f"[HNSW] Waktu: {t_hnsw:.3f} s")


# ---------------------------------------------------------
# 4. FAISS IVF-Flat
# ---------------------------------------------------------
def faiss_knn(data, k, nlist=100, nprobe=10):
    dim = data.shape[1]

    quantizer = faiss.IndexFlatL2(dim)
    index = faiss.IndexIVFFlat(quantizer, dim, nlist)
    index.train(data)
    index.add(data)
    index.nprobe = nprobe

    dist, idx = index.search(data, k)
    return idx


faiss_idx, t_faiss = timer(faiss_knn, X_norm, k)
print(f"[FAISS] Waktu: {t_faiss:.3f} s")


# ---------------------------------------------------------
# Optional: hitung tingkat keakuratan Approx vs Exact
# ---------------------------------------------------------
def recall_at_k(true_idx, approx_idx):
    total = 0
    correct = 0

    for i in range(len(true_idx)):
        gold = set(true_idx[i])
        pred = set(approx_idx[i])
        correct += len(gold.intersection(pred))
        total += len(gold)
    return correct / total


rec_annoy = recall_at_k(exact_idx, annoy_idx)
rec_hnsw = recall_at_k(exact_idx, hnsw_idx)
rec_faiss = recall_at_k(exact_idx, faiss_idx)

print("\nRecall@k (semakin tinggi semakin akurat):")
print(f"Annoy : {rec_annoy:.4f}")
print(f"HNSW  : {rec_hnsw:.4f}")
print(f"FAISS : {rec_faiss:.4f}")


# ---------------------------------------------------------
# Tampilkan Contoh Neighbor
# ---------------------------------------------------------
item = 0  # track pertama

print("\nTetangga terdekat (item pertama):")
print("Exact :", exact_idx[item][:5])
print("Annoy :", annoy_idx[item][:5])
print("HNSW  :", hnsw_idx[item][:5])
print("FAISS :", faiss_idx[item][:5])


100%|██████████| 894M/894M [00:09<00:00, 101MB/s] 

Extracting files...


[Exact] Waktu: 4623.783 s
[Annoy] Waktu: 145.218 s
[HNSW] Waktu: 366.529 s
[FAISS] Waktu: 836.581 s

Recall@k (semakin tinggi semakin akurat):
Annoy : 0.9486
HNSW  : 0.9955
FAISS : 0.9982

Tetangga terdekat (item pertama):
Exact : [     0 394553 764272 837727 749223]
Annoy : [0, 394553, 764272, 837727, 749223]
HNSW  : [     0 394553 764272 837727 749223]
FAISS : [     0 394553 764272 837727 749223]
